# TASK #3
Propose more examples of Data Quality checks for each DQ dimension to this dataset. Minimum 1 additional example per dimension.

In [1]:
import pandas as pd

In [2]:
df2016 = pd.read_csv('Parking_Violations_Issued_-_Fiscal_Year_2016.csv')

c:\users\orange\pycharmprojects\epam\venv\lib\site-packages\IPython\core\interactiveshell.py:3251: DtypeWarning: Columns (17,18,20,21,22,23,29,30,31,32,34,36,38,39) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10626899 entries, 0 to 10626898
Data columns (total 51 columns):
 #   Column                             Dtype  
---  ------                             -----  
 0   Summons Number                     int64  
 1   Plate ID                           object 
 2   Registration State                 object 
 3   Plate Type                         object 
 4   Issue Date                         object 
 5   Violation Code                     int64  
 6   Vehicle Body Type                  object 
 7   Vehicle Make                       object 
 8   Issuing Agency                     object 
 9   Street Code1                       int64  
 10  Street Code2                       int64  
 11  Street Code3                       int64  
 12  Vehicle Expiration Date            float64
 13  Violation Location                 float64
 14  Violation Precinct                 float64
 15  Issuer Precinct                    float64
 16  Issuer Code     

In [4]:
df2016.iloc[1,:]

Summons Number                          1363745293
Plate ID                                    KXD355
Registration State                              SC
Plate Type                                     PAS
Issue Date                              07/09/2015
Violation Code                                  21
Vehicle Body Type                             SUBN
Vehicle Make                                 CHEVR
Issuing Agency                                   P
Street Code1                                 55730
Street Code2                                 67030
Street Code3                                 58730
Vehicle Expiration Date                 20160288.0
Violation Location                            79.0
Violation Precinct                            79.0
Issuer Precinct                              301.0
Issuer Code                               358160.0
Issuer Command                                T301
Issuer Squad                                     0
Violation Time                 

### 3.1 Dimension 1 (Consistency)

Столбец 'Violation Code' должен соответствовать официально существующим кодам нарушений, размещенным на сайте  (https://data.ny.gov/widgets/ncbg-6agr) и не должен быть пустым

In [5]:
violation_codes = pd.read_csv('viol_codes.csv')
violation_codes.head()

,CODE,DEFINITION,Manhattan 96th St. & below,All Other Areas
0,1,FAILURE TO DISPLAY BUS PERMIT,515,515
1,2,NO OPERATOR NAM/ADD/PH DISPLAY,515,515
2,3,UNAUTHORIZED PASSENGER PICK-UP,515,515
3,4,BUS PARKING IN LOWER MANHATTAN,115,115
4,5,BUS LANE VIOLATION,50,50


In [6]:
df2016['Violation Code'].isin(violation_codes['CODE']).value_counts()

True     10625570
False        1329
Name: Violation Code, dtype: int64

In [29]:
df2016['Violation Code'].isnull().sum()

0

### 3.2 Dimension 2 (Timeliness)


Учитывая что данные в DF представлены за 2016 налоговый год, данные старше 2016-06-30 можно не принимать во внимание (тем более "даты из будущего" - даты старше даты формирования DF). Также могут быть неактуальны даты из прошлого столетия, но это будет зависить от целей обработки данных.

In [7]:
pd.to_datetime(df2016['Issue Date']).dt.year.value_counts().sort_index().head(10)

1970    2
1971    1
1973    4
1975    1
1979    2
1980    1
1982    1
1984    1
1985    2
1986    1
Name: Issue Date, dtype: int64

In [8]:
dd = df2016['Vehicle Expiration Date'][:5]
dd.astype('datetime64[ns]').dt.year

0    1970
1    1970
2    1970
3    1970
4    1970
Name: Vehicle Expiration Date, dtype: int64

In [9]:
df2016['Vehicle Expiration Date'].head(15)#.astype('int', errors='ignore')).head()#.dt.year.value_counts()

0     20170602.0
1     20160288.0
2            0.0
3            0.0
4     20160206.0
5     20160709.0
6     20160709.0
7     20170602.0
8     20150288.0
9     20170407.0
10           0.0
11    20160421.0
12    20150526.0
13           0.0
14    20160930.0
Name: Vehicle Expiration Date, dtype: float64

### 3.3 Dimension 3 (Completeness)


Столбцы 'Violation Code' и  'Violation Description' и 'House Number' и 'Street Name' попарно не должны быть пустыми для сохранения логической целостности данных

In [12]:
df2016[['Violation Code', 'Violation Description']].isna().sum()

Violation Code                 0
Violation Description    1139337
dtype: int64

In [13]:
df2016[['House Number', 'Street Name']].isna().sum()

House Number    2033420
Street Name        8274
dtype: int64

### 3.4 Dimension 4 (Uniqueness)

В DF существуют различные варианты написания цветов авто (сокращения, ошибки, описки)

In [14]:
df2016.groupby('Vehicle Color').agg({'Vehicle Color': 'count'}).rename(columns={'Vehicle Color':'count'}).sort_values(by='count', ascending=False).head(20).sort_index()

,count
Vehicle Color,
BK,1312004
BL,551057
BLACK,681848
BLK,77177
BLUE,284579
BR,86364
BROWN,298121
GR,213001
GRAY,67427


### 3.5 Dimension 5 (Validity)

Данные по смыслу являющиеся датой представлены "float64". 

In [15]:
df2016['Vehicle Expiration Date'].dtype

dtype('float64')

### 3.6 Dimension 6 (Accuracy)

Автомобильные номера должны соответствовать реально возможным значениям, длина номера обычно свыше 3 знаков и включает в себя числа и буквы.  
Исключить можно номера только из цифр, короче 3 знаков, а также пустые (nan) и без номеров ('BLANKPLATE')

In [16]:
df2016.groupby(by=df2016['Plate ID'].str.len())['Plate ID'].count()

Plate ID
1.0         131
2.0        1728
3.0        7942
4.0       13380
5.0      146069
6.0     1607971
7.0     8477259
8.0      352036
9.0         321
10.0      17898
Name: Plate ID, dtype: int64

In [18]:
df2016[~df2016['Plate ID'].str.contains('[A-z]',na=False, case=False, regex=True)][['Plate ID']].count()

Plate ID    196824
dtype: int64